In [1]:
!pip install -q streamlit

     |████████████████████████████████| 10.1 MB 8.4 MB/s 
     |████████████████████████████████| 77 kB 5.2 MB/s 
     |████████████████████████████████| 181 kB 52.6 MB/s 
     |████████████████████████████████| 4.3 MB 36.7 MB/s 
     |████████████████████████████████| 111 kB 41.4 MB/s 
     |████████████████████████████████| 164 kB 64.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 131 kB 74.0 MB/s 
     |████████████████████████████████| 428 kB 66.5 MB/s 
     |████████████████████████████████| 793 kB 50.5 MB/s 
     |████████████████████████████████| 130 kB 57.9 MB/s 
     |████████████████████████████████| 381 kB 66.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1

In [2]:
!pip install -q streamlit-ace

     |████████████████████████████████| 3.6 MB 9.6 MB/s 


In [3]:
!streamlit run app.py &>/dev/null&

In [4]:
!pip install -q pyngrok

     |████████████████████████████████| 745 kB 7.9 MB/s 


In [5]:
from pyngrok import ngrok

In [22]:
%%writefile app.py
import streamlit as st
import pandas as pd
from streamlit import session_state as _state
from joblib import dump, load

forest = load('RandomForestClassifierCyberSprint.joblib')
scaler = load('MinMaxScalerHGBoostCyberSprint.joblib')

fraudClientData = pd.read_excel('fraudClientData.xlsx')
deadClientData = pd.read_excel('deadClientData.xlsx')

fraudClientIds = fraudClientData['Id'].values.tolist()
deadClientIds = deadClientData['Id'].values.tolist()

blockedIds = fraudClientIds + deadClientIds

categoryOptions = {
    "Bars And Restaurants" : 0
    , "Contents" : 1
    , "Fashion" : 2
    , "Food" : 3
    , "Health" : 4
    , "Home" : 5
    , "Hotel Services" : 6
    , "Hyper" : 7
    , "Leisure" : 8
    , "Other Services" : 9
    , "Sports and Toys" : 10
    , "Tech" : 11
    , "Transportation" : 12
    , "Travel" : 13
    , "Wellness and Beauty" : 14
}

genderOptions = {
    'Empresa' : 0,
    'Feminino' : 1,
    'Masculino' : 2,
    'Não Identificar' : 3,
}

genderComboboxOptions = list(genderOptions.keys())
categoryComboboxOptions = list(categoryOptions.keys())

def convertDataToModel(state) :

  finalData = {
      'step' : [int(state.step)],
      'age' : [handleAgeSelection(int(state.age))],
      'gender': [masterSelectionHandler(state.gender, genderOptions)],
      'category' : [masterSelectionHandler(state.category, categoryOptions)],
      'amount' : [state.total]
  }

  df = pd.DataFrame(finalData)
  X_test = scaler.transform(df)

  return forest.predict(X_test)

def handlePayment(state) :

  if handleBlockedIds(int(state.id)):
    st.error('Pagamento Bloqueado.')
    return

  if convertDataToModel(state) == 0:
    st.success('Pagamento Aprovado.')
  else:
    st.warning('Pagamento Em Análise.')

def masterSelectionHandler(selectedData : str, data : dict) :
  return data.get(selectedData)

def handleBlockedIds(personId : int) -> bool:
  return personId in blockedIds

def handleAgeSelection(selectedAge : int) :
  
  age = 6

  if selectedAge <= 18:
    age = 0
  elif selectedAge <= 25:
    age = 1
  elif selectedAge <= 35:
    age = 2
  elif selectedAge <= 45:
    age = 3
  elif selectedAge <= 55:
    age = 4
  elif selectedAge <= 65:
    age = 5
  
  return age

def handleStateChangeToHome() :
  return st.session_state.update({"page": "home"})

################################################3

_PERSIST_STATE_KEY = f"{__name__}_PERSIST"

def persist(key: str) -> str:
    if _PERSIST_STATE_KEY not in _state:
        _state[_PERSIST_STATE_KEY] = set()

    _state[_PERSIST_STATE_KEY].add(key)

    return key


def load_widget_state():
    if _PERSIST_STATE_KEY in _state:
        _state.update({
            key: value
            for key, value in _state.items()
            if key in _state[_PERSIST_STATE_KEY]
        })

####################################

def main() :
  if "page" not in st.session_state:
    st.session_state.update({
      "page": "home",
      "name" : "",
      "gender" : "Empresa",
      "age" : "",
      "id" : "",
      "category" : "Bars And Restaurants",
      "step" : "",
      "total" : 0,
    })

  st.set_page_config(page_title="Carrinho", page_icon="🏷️", layout="centered")

  PAGES[st.session_state.page]()

def page_home() :
  st.title("🛒 Checkout")

  form = st.form(key="form")

  with form:
    name = st.text_input("Nome Completo:", key=persist("name"))
    colUserInfo = st.columns((2,1))
    gender = colUserInfo[0].selectbox("Gênero:", genderComboboxOptions, key=persist("gender"))
    age = colUserInfo[1].text_input("Idade:", key=persist("age"))
    id = st.text_input("Identificação:", key=persist("id"))
    productsCols1, productsCols2, productsCols3, productsCols4 = st.columns(4)

    st.markdown(""" 
                  <style>
                  @import url('https://fonts.googleapis.com/css?family=Space+Mono:400,400i,700,700i');
                  * {
                    font-family: 'Space Mono', monospace;
                  }
                  table {
                        width: 674px;
                        border-collapse: collapse; /*collapse = junta as bordas; sepparate = bordas separadas */
                        margin-bottom: 20px;
                      }
                      th, td  {
                        padding: 5px;
                      }
                      th {
                        text-align: left;
                        text-transform: uppercase;
                        letter-spacing: 0.1em;
                        font-size: 90%;
                        border-top: 1px solid #999;
                        border-bottom: 1px solid #111;
                      }
                      .numero_direita {
                        text-align: right;
                      }
                      tr:hover {
                        background-color: #c3e6e5;
                      }
                  </style>
                  <table>
                    <tr>
                      <th>Produto</th>
                      <th class="numero_direita">Quantidade</th>
                      <th class="numero_direita">Preço</th>
                    </tr>
                    <tr>
                      <td>Notebook</td>
                      <td class="numero_direita">1</td>
                      <td class="numero_direita">R$2.500</td>
                    </tr>
                    <tr class="row_cinza">
                      <td>Tablet</td>
                      <td class="numero_direita">2</td>
                      <td class="numero_direita">R$1.500</td>
                    </tr>
                    <tr>
                      <td>Iphone</td>
                      <td class="numero_direita">3</td>
                      <td class="numero_direita">R$3.500</td>
                    </tr>
                    <tr class="row_cinza">
                      <td>Nexus</td>
                      <td class="numero_direita">4</td>
                      <td class="numero_direita">R$3.500</td>
                    </tr>
                  </table>""", unsafe_allow_html=True)
    
    category = st.selectbox("Categoria:", categoryComboboxOptions, key=persist("category"))

    colf1, colf2, colf3 = st.columns(3)
    with colf2:
        pass
    with colf1:
        step = st.text_input('Step:', key=persist("step"))
    with colf3:
        total = st.number_input("Total:", key=persist("total"))

    colb1, colb2, colb3 , colb4, colb5 = st.columns(5)
    with colb1:
        pass
    with colb2:
        pass
    with colb4:
        pass
    with colb5:
        pass
    with colb3 :
        confirm = st.form_submit_button(label="Confirmar", on_click=st.session_state.update({"page": "payment"}))

def page_payment() :
  st.title("💳 Pagamento")

  form = st.form(key="form")

  with form:
    st.markdown("""
                <style>
                @import url('https://fonts.googleapis.com/css?family=Space+Mono:400,400i,700,700i');
                  *{
                    box-sizing:border-box;
                    font-family: 'Space Mono', monospace;
                  }
                  body, html{
                    margin:0;
                    padding:0;
                    height: 100%;
                    width: 100%;
                  }
                  body{
                    display: flex;
                    justify-content: center;
                    align-items: center;
                    display: -webkit-flex;
                    -webkit-align-items: center;
                    -webkit-justify-content: center;
                    background-image: linear-gradient(to right top, #a2beeb, #8ecfee, #8edde5, #a4e7d5, #c7eec7);
                    flex-direction: column;
                    -webkit-flex-direction: column;
                  }
                  .title {
                      margin-bottom: 30px;
                      color: #162969;
                  }
                  .card{
                  width: 320px;
                  height: 190px;
                    -webkit-perspective: 600px;
                    -moz-perspective: 600px;
                    perspective:600px;
                    margin: auto;
                    margin-bottom: 30px;
                    margin-top: 30px;
                  }
                  .card__part{
                    box-shadow: 1px 1px #aaa3a3;
                      top: 0;
                    position: absolute;
                  z-index: 1000;
                    left: 0;
                    display: inline-block;
                      width: 320px;
                      height: 190px;
                      background-image: url('https://image.ibb.co/bVnMrc/g3095.png'), linear-gradient(to right bottom, #fd696b, #fa616e, #f65871, #f15075, #ec4879); /*linear-gradient(to right bottom, #fd8369, #fc7870, #f96e78, #f56581, #ee5d8a)*/
                      background-repeat: no-repeat;
                      background-position: center;
                      background-size: cover;
                      border-radius: 8px;
                      -webkit-transition: all .5s cubic-bezier(0.175, 0.885, 0.32, 1.275);
                      -moz-transition: all .5s cubic-bezier(0.175, 0.885, 0.32, 1.275);
                      -ms-transition: all .5s cubic-bezier(0.175, 0.885, 0.32, 1.275);
                      -o-transition: all .5s cubic-bezier(0.175, 0.885, 0.32, 1.275);
                      transition: all .5s cubic-bezier(0.175, 0.885, 0.32, 1.275);
                      -webkit-transform-style: preserve-3d;
                      -moz-transform-style: preserve-3d;
                      -webkit-backface-visibility: hidden;
                      -moz-backface-visibility: hidden;
                  }
                  .card__front{
                    padding: 18px;
                  -webkit-transform: rotateY(0);
                  -moz-transform: rotateY(0);
                  }
                  .card__back {
                    padding: 18px 0;
                  -webkit-transform: rotateY(-180deg);
                  -moz-transform: rotateY(-180deg);
                  }
                  .card__black-line {
                      margin-top: 5px;
                      height: 38px;
                      background-color: #303030;
                  }
                  .card__logo {
                      height: 16px;
                  }
                  .card__front-logo{
                        position: absolute;
                      top: 18px;
                      right: 18px;
                  }
                  .card__square {
                      border-radius: 5px;
                      height: 30px;
                  }
                  .card_numer {
                      display: block;
                      width: 100%;
                      word-spacing: 4px;
                      font-size: 20px;
                      letter-spacing: 2px;
                      color: #fff;
                      text-align: center;
                      margin-bottom: 20px;
                      margin-top: 20px;
                  }
                  .card__space-75 {
                      width: 75%;
                      float: left;
                  }
                  .card__space-25 {
                      width: 25%;
                      float: left;
                  }
                  .card__label {
                      font-size: 10px;
                      text-transform: uppercase;
                      color: rgba(255,255,255,0.8);
                      letter-spacing: 1px;
                  }
                  .card__info {
                      margin-bottom: 0;
                      margin-top: 5px;
                      font-size: 16px;
                      line-height: 18px;
                      color: #fff;
                    letter-spacing: 1px;
                    text-transform: uppercase;
                  }
                  .card__back-content {
                      padding: 15px 15px 0;
                  }
                  .card__secret--last {
                      color: #303030;
                      text-align: right;
                      margin: 0;
                      font-size: 14px;
                  }
                  .card__secret {
                      padding: 5px 12px;
                      background-color: #fff;
                      position:relative;
                  }
                  .card__secret:before{
                    content:'';
                    position: absolute;
                  top: -3px;
                  left: -3px;
                  height: calc(100% + 6px);
                  width: calc(100% - 42px);
                  border-radius: 4px;
                    background: repeating-linear-gradient(45deg, #ededed, #ededed 5px, #f9f9f9 5px, #f9f9f9 10px);
                  }
                  .card__back-logo {
                      position: absolute;
                      bottom: 15px;
                      right: 15px;
                  }
                  .card__back-square {
                      position: absolute;
                      bottom: 15px;
                      left: 15px;
                  }
                  .card:hover .card__front {
                      -webkit-transform: rotateY(180deg);
                      -moz-transform: rotateY(180deg);

                  }
                  .card:hover .card__back {
                      -webkit-transform: rotateY(0deg);
                      -moz-transform: rotateY(0deg);
                  }
                </style>
                <div class="card">
                  <div class="card__front card__part">
                    <p class="card_numer">**** **** **** 7777</p>
                    <div class="card__space-75">
                      <span class="card__label">Títular do Cartão</span>
                      <p class="card__info">CAROLINNE PINHEIRO</p>
                    </div>
                    <div class="card__space-25">
                      <span class="card__label">Validade</span>
                            <p class="card__info">10/25</p>
                    </div>
                    </div>
                    <div class="card__back card__part">
                      <div class="card__black-line"></div>
                      <div class="card__back-content">
                        <div class="card__secret">
                          <p class="card__secret--last">777</p>
                        </div>
                      </div>
                  </div>
                </div>
                """, unsafe_allow_html=True)

    cardHolderName = st.text_input("Nome Completo no Cartão:")

    colp1, colp2, colp3, colp4 = st.columns(4)

    with colp1:
      cardHolderNumberOne = st.text_input(label='Número do Cartão:', key='cardNumberOne', max_chars=4)
    with colp2:
      cardHolderNumberTwo = st.text_input(label='', key='cardNumberTwo', max_chars=4)
    with colp3:
      cardHolderNumberThree = st.text_input(label='', key='cardNumberThree', max_chars=4)
    with colp4:
      cardHolderNumberFour = st.text_input(label='', key='cardNumberFour', max_chars=4)

    cole1, cole2, cole3, cole4 = st.columns(4)

    with cole1:
      cardExpireDay = st.text_input(label='Mês Expiração:', key='cardExNumberOne', max_chars=2)
    with cole2:
      cardExpireYear = st.text_input(label='Ano Expiração:', key='cardExNumberTwo', max_chars=4)
    with cole3:
      pass
    with cole4:
      cardCVC = st.text_input(label='CVC:', key='cardExNumberThree', max_chars=3)

    colb1, colb2 , colb3 = st.columns(3)

    with colb1:
        backToHome = st.form_submit_button(label="Retornar")
    with colb2:
        pass
    with colb3:
        submitted = st.form_submit_button(label="Confirmar Pagamento")
    
    if backToHome:
      handleStateChangeToHome()
    if submitted:
      handlePayment(st.session_state)

PAGES = {
    "home": page_home,
    "payment": page_payment,
}

if __name__ == "__main__":
  load_widget_state()
  main()

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-05-15 20:14:20.070 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.664s
your url is: https://wild-ends-pay-104-199-123-139.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://104.199.123.139:8501

